In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# Define a structured prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n"),
    ("human", "{equation_statement}")
])

# Initialize the Ollama model
model = ChatOllama(model="llama3.2", temperature=0)

# Create the runnable chain
runnable = (
    {"equation_statement": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

# Invoke the chain
print(runnable.invoke("x raised to the third plus seven equals 12"))


In [ ]:
# Bind a stop sequence to prevent output after the equation
runnable = (
    {"equation_statement": RunnablePassthrough()}
    | prompt
    | model.bind(stop=["SOLUTION"])  # Corrected: Stop expects a list
    | StrOutputParser()
)

# Invoke the chain
print(runnable.invoke("x raised to the third plus seven equals 12"))


In [ ]:
# Define a tool for retrieving weather data
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

# Bind the tools to the model
model_with_tools = ChatOllama(model="llama3.2").bind(tools=tools)

# Invoke the model
print(model_with_tools.invoke("What's the weather in SF, NYC and LA?"))
